# 텐서플로로 분산 학습하기
---

tf.distribute.Strategy는 훈련을 여러 GPU, TPU등으로 나누어 처리하기 위한 API입니다.

이 API를 사용하면, 기존의 모델이나 훈련 코드를 조금만 고쳐서 분산 학습을 할 수있습니다.

이 API는 다음과 같은 목표로 설계되었습니다.

- 사용하기 쉽고, 연구원, 기계학습 엔지니어 등 여러 사용자 층을 지원할 것
- 그대로 적용하기만 하면 좋은 성능을 보일 것
- 전략들을 쉽게 바꿀 수 있을 것

tf.distribute.Strategy는 High Level API인, tf.keras, tf.estimator와 함께 사용할 수 있습니다.

코드를 한두줄만 추가하면 됩니다.

그리고 사용자 정의 학습에서도 사용할 수 있도록 제공하고 있습니다.

텐서플로 2.0에서는 Eager Execution 할수도, tf.function을 사용하여 그래프에서 실행할 수도 있습니다.

Document에 따라서, 지금은 훈련에 대해 이야기만 하지만, Inference, Prediction 등에도 활용할 수 있습니다.

다시한번 작성하지만, 기존의 코드에서 약간만 바꾸면 tf.distribute.Strategy를 사용할 수 있습니다.

모든 변수, 레이어, 모델, 옵티마이저, 지표, Summary, CheckPoint등 텐서플로를 구성하고 있는 기반 요소들은 Strategy를 이해하고, 처리할 수 있도록 수정했다고 합니다.

이제 여러 상황에서 이들을 어떻게 사용해야하는지 알아봅니다.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Strategy의 종류
---

tensorflow2.4.0버젼 기준으로 현재 공식적으로 지원하고 있는 종류는 다음과 같습니다. 

추후에 추가되기도 하고, 사라지기도 할 예정입니다.

현재 지원한다고 해도 기본적으로는 대부분 실험 중입니다.

이들 중 몇가지를 살펴보기로하겠습니다.

- 동기 훈련 대 비동기 훈련: 분산 훈련을 할 때 데이터를 병렬로 처리하는 방법은 크게 두 가지가 있습니다. 동기훈련을 할 때는 모든 워커가 입력 데이털르 나누어 갖고 동시에 훈련합니다. 그리고 각 단계 마다 그래디언트를 수집합니다. 비동기 훈련에서는 모든 워커가 독립적으로 입력데이털르 사용해 훈련하고, 각각 비동기적으로 변수들을 갱신합니다. 일반적으로 동기 훈련은 올 리듀스 방식으로 구현하고, 비동기 훈련은 파라미터 서버 구조를 사용합니다.

- 하드웨어 플랫폼: 한 장비에 있는 다중 GPU로 나누어 훈련할 수도, 네트워크로 연결된 여러 장비로 나누어서 훈련할수도, 혹은 클라우드에서 다중 TPU로 학습할 수도 있습니다.

총 6가지 전략을 공식적으로 지원하고 있습니다.

훈련 API | MirroredStrategy | TPUStrategy | MultiWorkerMirroredStrategy | CentralStorageStrategy | ParameterServerStrategy
---| :---: | :---: | :---: | :---: | :---: 
Keras API | 지원 | 지원 | 실험 기능으로 지원 | 실험 기능으로 지원 | 실험 기능으로 지원
사용자 정의 훈련 루프 | 지원 | 지원 | 실험 기능으로 지원 | 실험 기능으로 지원 | 실험 기능으로 지원
Estimator API | 제한적으로 지원 | 미지원 | 제한적으로 지원 | 제한적으로 지원 | 제한적으로 지원

# MirroredStrategy
---

tf.distribute.MirroredStratgy는 장비 하나에서 다중 GPU를 이용한 동기 분산훈련을 지원합니다.

각 GPU마다 복제본이 만들어집니다.

모델의 변수는 복제본마다 미러링 됩니다.

미러링된 변수들은 하나의 가상 변수에 대응되는데 이 변수를 MirroredVariable이라고합니다.

이 변수들은 동일한 변경사항이 함께 적용되므로 항상 같은 값을 유지합니다.

여러 장치에 변수의 변경사항을 전달하기 위하여 효율적인 All-Reduce 알고리즘을 사용합니다.

All-Reduce 알고리즘은 모든 장치에 걸쳐 텐서를 모은 다음, 그 합을 구하여 다시 각 장비에 제공합니다.

이 알고리즘은 효율적이여서 동기화의 부담을 많이 덜어낼 수 있습니다.

장치간의 사용 가능한 통신 방법에 따라 다양한 All-Reduce 알고리즘과 구현이 있습니다.

기본값으로는 NVIDIA NCCL을 All-Reduce 구현으로 사용합니다.

다른 방법으로는 제공되는 다른 방식을 선택하거나, 직접구현할 수 있습니다.

MirroedStrategy를 사용하는 방법은 다음과 같습니다.

In [2]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


일부만 사용하고 싶다면 다음과 같이 진행 할 수 있습니다.

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


장치간의 통신 방법을 바꾸고 싶다면, cross_device_ops인자에, tf.distribute.CrossDeviceOps 타입의 인스턴스를 넘기면 됩니다.

제공되는 방식은 다음과 같습니다.

- tf.distribute.NcllAllReduce
- tf.distribute.HierarchicalCopyAllReduce
- tf.distribute.ReductionToOneDevice

In [4]:
mirrored_strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [5]:
mirrored_strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.ReductionToOneDevice)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


# CentralStorageStrategy
---

tf.distribute.experimental.CentralStorageStrategy도 동기 훈련을 합니다.

하지만 변수를 미러링하지 않고, CPU에서 관리합니다.

작업은 모든 로컬 GPU들로 복제됩니다. 만약에 GPU가 하나 밖에 없다면, 모든 변수와작어비 그 GPU에 배치됩니다.

In [6]:
central_storage_strategy = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1'], variable_device = '/device:CPU:0'


# MultiWorkerMirroredStrategy

tf.distribute.experimental.MultiWorkerMirroredStrategy는 MirroredStrategy와 매우 비슷합니다.

다중 워커를 사용하여 동기 분산 훈련을 진행합니다.

각 워커는 여러개의 GPU를 사용할 수 있습니다. MirroredStrategy처럼 모델에 있는 모든 변수의 복사본을 모든 워커의 각 장치에 만듭니다.

다중 워커들 사이에서는 all-reduce 통신 방법으로 CollectiveOps를 사용하여 변수들을 같은 값으로 유지합니다.

Collective Operations은 텐서플로 그래프에 속하는 연산중 하나입니다.

이 연산은 하드웨어나 네트워크 구성 텐서 크기에 따라 텐서플로 런타임이 지우너하는 all-reduce 알고리즘을 자동으로 선택합니다.

여기에 추가 성능 최적화기능도 구현하고 있습니다.

예를들면, 작은 텐서들의 여러 all-reduce작업을 큰 텐서들의 더 적은 all-reduce작업으로 바꾸는 최적화 기능이 있습니다.

뿐만 아니라 pulgin 구조를 갖도록 설계했습니다.

추후에는 사용자가 하드웨어에 더 최적화된 알고리즘을 사용할수도 있을 것입니다.

이 연산은 all-reduce외에 브로드 캐스트나 all-gather도 구현하고 있습니다.

In [7]:
multiworker_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1'), communication = CollectiveCommunication.AUTO


MultiWorkerMirroredStrategy에 사용할 수 잇는 수집 연산은 현재 두가지를 지원하고 있습니다.

- CollectiveCommunication.RING : gRPC를 사용한 링 네트워크 기반의 수집 연산입니다.
- CollectiveCommunication.NCCL : Nvidia의 NCCL을 사용하여 수집 연산을 구현한 것입니다.
- CollectiveCommunication.AUTO : 런타임이 알아서 구현을 고릅니다. 최적화는 GPU수와 종류, 클러스터의 네트워크 연결등에 따라 다를 수 있습니다.

In [8]:
multiworker_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
    tf.distribute.experimental.CollectiveCommunication.NCCL )

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1'), communication = CollectiveCommunication.NCCL


# 다중 GPU VS 다중 Worker
---

이 둘의 차이에 대해 이야기를 하면 다중 워커의 설정하는 방법 다릅니다.

클러스터를 구성하는 각워커에 TF_CONFIG환경 변수를 사용하여 클러스터 설정을 하는 것이 텐서플로의 표준 방법입니다.

# TPUStrategy
---

tf.distribute.experimental.TPUStrategy는 TPU에서 수행하는 전략입니다.

분산 훈련의 구조 측면에서 MirroredStrategy와 동일합니다.

In [9]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
    tpu=tpu_address)
tf.config.experimental_connect_to_host(cluster_resolver.master())
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)

NameError: name 'tpu_address' is not defined

# PrameterServerStrategy
---

tf.distribute.experimental.PrameterServerStrategy는 여러 장비에서 훈련할 때 파라미터 서버를 사용합니다.

이 전략을 사용하면 몇대의 장비는 워커 역할을 하고, 몇대는 파라미터 서버 역할을 하게 됩니다.

모델의 각 변수는 한 파라미터 서버에 할당됩니다.

계산작업은 모든 워커 GPU들에 복사됩니다.

코드만 놓고 봤을 땐, 다른 전략들과 비슷합니다.

In [10]:
ps_strategy = tf.distribute.experimental.PrameterServerStrategy()

AttributeError: module 'tensorflow._api.v2.distribute.experimental' has no attribute 'PrameterServerStrategy'

다중 워커 환경에서 훈련하려면, 클러스터에 속한 파라미터 서버와 워커를 "TF_CONFIG" 환경변수를 이용하여 설정해야합니다.

# Using tf.distribute.Strategy with Keras
---

tf.keras는 분산전략을 백엔드와 함께 쓸 수 있으므로, 쉽게 분산 처리할 수 있습니다.

일부만 수정함으로 금방 분산 처리를 할 수 있습니다.

고쳐야하는 부분은 다음과 같습니다.

1. 적절한 tf.distribute.Strategy 인스턴스 생성
2. 케라스 모델의 생성과 컴파일을 strategy.scope안으로 옮겨주기

Sequential, Functional, SubClassing 모두 지원합니다.

In [11]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])
    model.compile(loss='mse', optimizer='sgd')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [12]:
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(10)
model.fit(dataset, epochs=2)
model.evaluate(dataset)

Epoch 1/2
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
10/10 [==============================] - 0s 2ms/step - loss: 1.2136
Epoch 2/2
10/10 [==============================] - 0s 2ms/step - loss: 0.5364
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:

0.3331356346607208

당연하겠지만, 넘파이를 이용해 학습하는 것도 가능합니다.

In [13]:
inputs, targets = np.ones((100, 1)), np.ones((100, 1))
model.fit(inputs, targets, epochs=2, batch_size=10)

Epoch 1/2
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
10/10 [==============================] - 0s 2ms/step - loss: 0.2371
Epoch 2/2
10/10 [==============================] - 0s 3ms/step - loss: 0.1048


데이터셋이나 넘파이를 사용하는 두 경우 모두 입력 배치가 동일한 크기로 나누어져서 여러개로 복제된 작업에 전달됩니다.

예를 들어 MirroredStrategy의 경우 2개의 GPU에서 사용한다면, 10 Batch시 5 Batch씩 두개로 나누어 학습됩니다.

따라서 GPU가 추가될 수록 각 Epoch당 훈련 시간은 줄어들게 됩니다.

일반적으로 GPU등이 추가 될 때마다 추가한 컴퓨팅 자원을 더 효과적으로 사용하기 위해 배치사이즈도 더 키웁니다.

모델에 따라서는 Learning rte를 재조정 해야할 수도 있습니다.

복제본의 수는 strategy.num_replicase_in_sync로 얻을 수 이습니다.

In [14]:
# 복제본 사이즈로 전체 배치 크기얻기
BATCH_SIZE_PER_REPLICA = 5
global_batch_size = (BATCH_SIZE_PER_REPLICA *
                     mirrored_strategy.num_replicas_in_sync)
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100)
dataset = dataset.batch(global_batch_size)

LEARNING_RATES_BY_BATCH_SIZE = {5: 0.1, 10: 0.15}
learning_rate = LEARNING_RATES_BY_BATCH_SIZE[global_batch_size]

# Using tf.distribute.Strategy with Custom loop
---

High Level API를 쓰지 않는 사용자도, 조금만 더 노력을 들이면 사용할 수 있습니다.

tf.distribute.Strategy 클래스는 여러 메서드를 제공합니다.

우선 전략의 범위를 설정합니다.

In [15]:
with mirrored_strategy.scope():
    model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])
    optimizer = tf.keras.optimizers.SGD()

다음은 tf.distribute.Strategy.experimental_distribute_dataset 메서드를 호출하여 전략에 맞게 데이터셋을 분배합니다.

In [16]:
with mirrored_strategy.scope():
    dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(1000).batch(global_batch_size)
    dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

그리고 한단계의 훈련을 정의합니다.

분산훈련을 위해 tf.distribute.Strategy.experimental_run_v2 메서드로 전달합니다.

In [17]:
@tf.function
def train_step(dist_inputs):
    def step_fn(inputs):
        features, labels = inputs
        
        with tf.GradientTape() as tape:
            logits = model(features)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            loss = tf.reduce_sum(cross_entropy) * (1.0 / global_batch_size)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
        return cross_entropy

    per_example_losses = mirrored_strategy.experimental_run_v2(step_fn, args=(dist_inputs,))
    mean_loss = mirrored_strategy.reduce(tf.distribute.ReduceOp.MEAN, per_example_losses, axis=0)
    return mean_loss